In [ ]:
from tnmpa import KsatInstance, BeliefPropagation

In [ ]:
from quimb.tensor import COPY_tensor
import quimb.tensor as qtn

In [ ]:
import random
import numpy as np

In [ ]:
# paramter the controls the complexity of the problem, ratio between number of clauses and number of variables
# for alpha > 3.5 BP should start to fail
alpha = 2.0

# number of variables
N = 30

# numer of clauses
M = int(alpha * N)

# K of K-SAT
K = 3

In [ ]:
seed = 0
instance = KsatInstance(N, M, K, seed=seed)

In [ ]:
ksol_sp = BeliefPropagation(instance)
status = ksol_sp.MP(ksol_sp.envs_tensors, ksol_sp.instance, tol=1e-13)
status["bp_converged"], status["max_distance"]

In [ ]:
ksol_sp.entropy / N

In [ ]:
tn = instance.tn(factor_threshold=6)
tn.draw(color=["VARIABLE", "CLAUSE"])

In [ ]:
tn.outer_inds()

In [ ]:
for v in range(N):
    if f"T{v}" not in tn.tag_map.keys():
        continue
    tn, tt = tn.partition_tensors(f"T{v}")
    if len(tn.outer_inds()) > 1:
        new_inds = {i: f"p{v}" for i in tn.outer_inds()}
        tn.reindex(new_inds, inplace=True)
    else:
        tn |= tt

In [ ]:
tn.draw(color=["VARIABLE", "CLAUSE"])

In [ ]:
distances = []
print(f"var\tL1 norm")
for v in range(N):
    tnv = tn.copy(deep=True)
    if f"p{v}" in tnv.ind_map.keys():
        tids = tnv.ind_map[f"p{v}"].copy()
        for k, tid in enumerate(tids):
            t = tnv.tensor_map[tid]
            t.reindex({f"p{v}":f"p{v}_{k}"}, inplace=True)
    elif f"T{v}" in tnv.tag_map.keys():
        tnv, tt = tnv.partition_tensors(f"T{v}")
    else:
        continue
    
    tnv |= qtn.tensor_core.COPY_mps_tensors(2, list(tnv.outer_inds())+["out"])
    
    for _ in range(1):
        tnv = tnv.full_simplify()
    
    tout = tnv.contract(output_inds=("out",), optimize="greedy")
    mv0 = tout.data[0]
    mv1 = tout.data[1]
    exact = np.array([mv0 / (mv1 + mv0),  mv1 / (mv1 + mv0)])
    
    bp = np.array(ksol_sp.variable_marginal(v))
    distances.append(sum(abs(exact - bp)))
    print(f"{v}\t{sum(abs(exact - bp))}")

In [ ]:
np.mean(distances)